In [ ]:
import numpy as np

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.utils import bgr8_to_jpeg
from jetcam.imx219 import IMX219

camera = IMX219(capture_width=1280, capture_height=720, fps=30, width=224, height=224)
racecar = DefaultRacecar()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
display(image_widget)

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image

# define model
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_circuitlaunch_xy.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval()

# define preprocessing
mean = torch.Tensor([0.485, 0.456, 0.406]).float().cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).float().cuda()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).float()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
controller = ipywidgets.Controller(index=0)
display(controller)

In [ ]:
# control sliders
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.33, description='steering gain')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=-0.10, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_bias_slider)

In [ ]:
# display sliders
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

In [ ]:
def execute(change):
    image = change['new']
    xy = model(preprocess(image)).detach().cpu().numpy().flatten()
    
    x = xy[0]
    y = 1.0 - xy[1]
    x_slider.value = x
    y_slider.value = y
    angle = np.arctan2(x, y)
    
    speed_slider.value = speed_gain_slider.value
    steering_slider.value = x_slider.value * steering_gain_slider.value + steering_bias_slider.value

    racecar.steering = steering_slider.value
    
    if bool(controller.buttons[5].value):
        racecar.throttle = speed_slider.value
    else:
        racecar.throttle = 0.0

In [ ]:
import numpy as np

In [ ]:
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
camera.unobserve(execute, names='value')

In [ ]:
camera.unobserve_all()